In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
from sklearn import tree


In [3]:
# loading the data from the csv file into a data frame
file_path = r"C:\Users\raaja\OneDrive\Documents\maichu development\notebooks\merged_data_with_labels.csv"
df = pd.read_csv(file_path)

desired_columns = ["windspeedKmph", "winddirDegree", "precipMM", "visibility", "pressure", "cloudcover", "DewPointF", "WindGustKmph", "tempF", "WindChillF", "humidity", "DepDelayMinutes", "IsDelayed"]


desired_columns = ["windspeedKmph", "winddirDegree", "precipMM", "visibility", "pressure", "cloudcover", "DewPointF", "WindGustKmph", "tempF", "WindChillF", "humidity", "DepDelayMinutes", "IsDelayed"]

df = df.drop(columns=list(set(df.columns) - set(desired_columns)))

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.drop("IsDelayed", axis=1), df["IsDelayed"], test_size=0.2, random_state=42)

clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
    max_depth=1, random_state=0).fit(X_train, y_train)
 


In [4]:

clf.score(X_test, y_test)

0.8120949099079635

In [5]:
# make predictions on the test set
y_pred = clf.predict(X_test)
 
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
confusion_m = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)
 
# Print the results
print(f"Accuracy: {accuracy:.2f}")
print("\nConfusion Matric:\n", confusion_m)
print("\nClassification Report:\n", classification_rep)

Accuracy: 0.81

Confusion Matric:
 [[ 82515  57203]
 [ 12376 218194]]

Classification Report:
               precision    recall  f1-score   support

     Delayed       0.87      0.59      0.70    139718
 Not Delayed       0.79      0.95      0.86    230570

    accuracy                           0.81    370288
   macro avg       0.83      0.77      0.78    370288
weighted avg       0.82      0.81      0.80    370288



In [6]:
import os
import csv

def flatten_classification_report(report):
    flattened_report = {}
    for key, value in report.items():
        if isinstance(value, dict):
            for sub_key, sub_value in value.items():
                flattened_report[f"{key}_{sub_key}"] = sub_value
        else:
            flattened_report[key] = value
    return flattened_report

def evaluate_and_save_classification_results(model_name, y_test, y_pred, output_file="classification_results.csv"):
    confusion_matrix_result = confusion_matrix(y_test, y_pred)
    classification_report_result = classification_report(y_test, y_pred, output_dict=True)

    
    # extract features from confusion matrix
    tn, fp, fn, tp = confusion_matrix_result.ravel()
    
    # flatten the classification report
    flattened_report = flatten_classification_report(classification_report_result)
    
    # add confusion matrix metrics to the flattened report
    flattened_report['confusion_matrix_tn'] = tn
    flattened_report['confusion_matrix_fp'] = fp
    flattened_report['confusion_matrix_fn'] = fn
    flattened_report['confusion_matrix_tp'] = tp
    
    # include the model name as a feature
    flattened_report = {'model_name': model_name, **flattened_report}

    # write the flattened data to a csv file
    file_exists = os.path.isfile(output_file)
    
    with open(output_file, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=flattened_report.keys())
        # write the header only if the file does not exist
        if not file_exists:
            writer.writeheader()
        # write the data
        writer.writerow(flattened_report)

    print(f"Metrics have been saved to {output_file}")
    
evaluate_and_save_classification_results("XGBoost", y_test, y_pred)


Metrics have been saved to classification_results.csv
